## Supervised Machine Learning Models - Part 2

### Table of Contents

* [PART 1](#chapter1)  

    * [Ensemble Learning algorithms](#section_1_1)
        1. [Bagging algorithms](#Section_1_1_1)
            *  [Random Forest](#section_2_1_1)
        3. [Boosting algorithms](#section_3_1_1)
             * [AdaBoost](#section_3_2_1)
             * [Gradient Boosting XGBoost ](#section_3_2_2)
             * [XGBoost ](#section_3_2_3)
        4. [Stacking](#section_4_1_1)
<br>

* [PART 2](#chapter2)

    * [Cross Validation](#section_4_1)
    * [HyperParameter Tuning](#section_5_1)
        * [GridsearchCV](#section_5_1_1)
        * [RandomSearchCV](#section_5_1_2)
    * [Model Comparison](#section_6_1)

## PART 1 <a class="anchor" id="chapter1"></a>

## Ensemble Learning algorithms <a class="anchor" id="section_1_1"></a>

Ensemble learning models are simply combinations of different machine learning models.Instead of training one large/complex model for your dataset, you train multiple small/simpler models (weak-learners) and aggregate their output (in various ways) to form your prediction.

It combines multiple weak models/learners into one predictive model to reduce bias, variance and/or improve accuracy.


<font color='Brown'>**Types of Ensemble Learning: N number of weak learners:**</font>
1. Bagging
2. Boosting
3. Stacking

These methods have the same wisdom-of-the-crowd concept but differ in the details of what it focuses on, the type of weak learners used, and the type of aggregation used to form the final output.





### 1. Bagging algorithms <a class="anchor" id="Section_1_1_1"></a>

**Bagging (Boostrap Aggregating):** Trains N different weak models (usually of same types – homogenous) with N non-overlapping subset of the input dataset **in parallel.** In the test phase, each model is evaluated. The label with the greatest number of predictions is selected as the prediction. Bagging methods reduces variance of the prediction.

For each weak-learner, the input data is randomly sampled from the original dataset with replacement and is trained. By sampling with replacement some observations may be repeated in each new training data set. A random sampling of the subset with replacement creates nearly iid samples. During inference, the test input is fed to all the weak-learners and the output is collected. The result outputted from bagging is the average (if the problem is regression) or the most suitable label by the voting scheme (if the problem is classification).

In bagging methods, the weak-learners, usually are of the same type. Since the random sampling with replacement creates iid samples, and aggregating iid variables doesn’t change the bias but reduces variance, the bagging method doesn't change the bias in the prediction but **reduces its variance.**

<img src="Images/bag.png" width="300"/>

Each decision tree in the bag is trained on an independent subset of the training data. These subsets are random bootstraps of the whole training set. In other words, suppose the training data is a table with n observations on m features. Each component tree of the bagging will receive a subset of k observations on m features to train on, with k < n. Each observation of a subset is drawn from the full data with replacement.

In [ ]:
from sklearn.ensemble import BaggingRegressor
bag = BaggingRegressor(base_estimator=DecisionTreeRegressor()) 

#### Random Forest <a class="anchor" id="section_2_1_1"> </a>

- Random forest is very similar to bagging: it also consists of many decision trees, each of the trees is assigned with a bootstrap sample of the training data, and the final result of the meta-model is computed as the average or mode of the outputs from the components.

- The only difference is that random forests, when splitting a node of a component tree, not all of the features are taken as candidates to split. Instead, only a subset of the whole feature set is selected to be the candidates (the selection is random for each node) and then the best feature from this subset is appointed to be the splitting test at that node. Suppose there are m features overall, the size of the subset can be any number from 1 to m-1.

- The random forests algorithm tries to decorrelate the trees so that they learn different things about the data. It does this by selecting **a random subset of variables.** If one or a few independent variables are very strong predictors for the response variable, these features will be selected in many of the trees, causing them to become correlated. Random subsampling of independent variables ensures that not always the best predictors overall are selected for every tree and, the model does have a chance to learn other features of the data.

- They are simply ensembles of decision trees. Each tree is trained with a different randomly selected part of the data with randomly selected independent variables. The goal of introducing randomness is **to reduce the variance** of the model so it does **not overfit**, at the expense of a **small increase in the bias** and **some loss of interpretability**. This strategy generally boosts the performance of the final model.

**How to:**

- Individual trees are built independently, using the same procedure as for a normal decision tree but with only a random portion of the data and only considering a random subset of the features at each node. Aside from this, the training procedure is exactly the same as for an individual Decision Tree, repeated N times.

- To make a prediction using a Random Forest each an individual prediction is obtained from each tree. Then, if it is a classification problem, we take the most frequent prediction as the result, and if it is a regression problem we take the average prediction from all the individual trees as the output value. The following figure illustrates how this is done:


<img src="Images/randomforest.png" width="700"/>

**Parameters:**
For random forests, we have two critical arguments. 

- Number of Features: how many columns to use when sampling 
- Number of Trees: how many trees to average




In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr= RandomForestRegressor()

### 2. Boosting algorithms <a class="anchor" id="section_3_1_1"></a>

The general idea of boosting also encompasses building multiple weak learners to contribute to the final result. However, these component trees are built **sequentially**, one after another, and how to build the latter one is dependent on the result of the formers. Put another way, the next weak learner is built in a way to specifically improve on what the existing weak learners are bad at.

There are a few answers to how should the next tree address the shortcomings of the previous trees, these answers divide boosting into several styles. The most popular styles are **AdaBoost**, **Gradient Boosting** and **XGBoost**

In the test phase, each model is evaluated and based on the test error of each weak model, the prediction is weighted for voting. Boosting methods **decreases the bias of the prediction.**

<img src="Images/boost.png" width="300"/>

Step 1:  The base learner takes all the distributions and assign equal weight or attention to each observation.

Step 2: If there is any prediction error caused by first base learning algorithm, then we pay higher attention to observations having prediction error. Then, we apply the next base learning algorithm.

Step 3: Iterate Step 2 till the limit of base learning algorithm is reached or higher accuracy is achieved.

Finally, it combines the outputs from weak learner and creates  a strong learner which eventually improves the prediction power of the model. Boosting pays higher focus on examples which are mis-classiﬁed or have higher errors by preceding weak rules.





#### AdaBoost (Adaptive Boosting) <a class="anchor" id="section_3_2_1"></a>

Adaptive Boosting aka AdaBoost fits a sequence of weak learners on different weighted training data. It starts by predicting original data set and gives equal weight to each observation. If prediction is incorrect using the first learner, then it gives higher weight to observation which have been predicted incorrectly. Being an iterative process, it continues to add learner(s) until a limit is reached in the number of models or accuracy.

**How:**
<img src="Images/ada.png" width="500"/>

***Box 1:*** You can see that we have assigned equal weights to each data point and applied a decision stump to classify them as + (plus) or – (minus). The decision stump (D1) has generated vertical line at left side to classify the data points. We see that, this vertical line has incorrectly predicted three + (plus) as – (minus). In such case, we’ll assign higher weights to these three + (plus) and apply another decision stump.

***Box 2:*** Here, you can see that the size of three incorrectly predicted + (plus) is bigger as compared to rest of the data points. In this case, the second decision stump (D2) will try to predict them correctly. Now, a vertical line (D2) at right side of this box has classified three mis-classified + (plus) correctly. But again, it has caused mis-classification errors. This time with three -(minus). Again, we will assign higher weight to three – (minus) and apply another decision stump.

***Box 3:*** Here, three – (minus) are given higher weights. A decision stump (D3) is applied to predict these mis-classified observation correctly. This time a horizontal line is generated to classify + (plus) and – (minus) based on higher weight of mis-classified observation.

***Box 4:*** Here, we have combined D1, D2 and D3 to form a strong prediction having complex rule as compared to individual weak learner. You can see that this algorithm has classified these observation quite well as compared to any of individual weak learner.

We can use AdaBoost algorithms for both classification and regression problem.

***The drawback of AdaBoost*** is that it is easily defeated by noisy data, the efficiency of the algorithm is highly affected by outliers as the algorithm tries to fit every point perfectly.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier #For Classification
from sklearn.ensemble import AdaBoostRegressor #For Regression
clf = AdaBoostClassifier(n_estimators = 50, base_estimator = DecisionTreeClassifier)
clf.fit(x_train,y_train)
clf.predict(x_test)

**Parameters**
- N estimators: It controls the number of weak learners.
- Learning Rate: Controls the contribution of weak learners in the final combination. There is a trade-off between learning_rate and n_estimators.
- Base estimators: It helps to specify different ML algorithm.


#### Gradient Boosting <a class="anchor" id="section_3_2_2"></a>

Gradient boosting is also based on the sequential and symbol learning model. The base learners are generated sequentially in such a way that the present based learner is always more effective than the previous one. The overall model improves sequentially with each iteration now.

However, in this boosting the weights for misclassified outcomes are not incremented. The main idea here is to overcome the residual errors (prediction error for the current ensemble of models) of the previous model. It tries to optimize the loss function of previous learner by adding a new adaptive model that combines weak learners. Specifically in gradient boosting, the simple models are trees. As in random forests, many trees are grown but in this case, trees are sequentially grown and each tree focuses on fixing the shortcomings of the previous trees. 

<img src="Images/gb.png" width="500"/>

Gradient boosting is a greedy algorithm and can overfit a training dataset quickly.

The loss function is the one that needs to be optimized (Reduce the error) You have to keep adding a model that will regularize the loss function from the previous learner.
Just like adaptive boosting gradient boosting can also be used for both classification and regression.

In [3]:
from sklearn.ensemble import GradientBoostingClassifier #For Classification
from sklearn.ensemble import GradientBoostingRegressor #For Regression
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, loss_function = deviance, max_depth=1)
clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined

**Parameters**
- N estimators: It controls the number of weak learners.
- Learning Rate: Controls the contribution of weak learners in the final combination. There is a trade-off between learning_rate and n_estimators.
- Loss Reduction: The loss function to use during splitting
- Sample size: The proportion of the data exposed to the model during each iteration.
- Max_depth: maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree.


#### XG Boosting <a class="anchor" id="section_3_2_3"></a>

- One of the most widely used algorithms for gradient boosting is XGBoost which stands for “extreme gradient boosting” which is the a more advanced version of the gradient boosting method. XGboost as well as other gradient boosting methods has many parameters to regularize and optimize the complexity of the model. This flexibility comes with benefits; methods depending on XGboost have won many machine learning competitions.

- XGBoost was introduced because the gradient boosting algorithm was computing the output at a prolonged rate right because there's a sequential analysis of the data set and it takes a longer time.

- The main aim of this algorithm is to increase **speed** and model **performance**.
- It supports **parallelization** by creating decision trees. There's no sequential modeling in computing methods for evaluating any large and any complex modules.

- Around the time it was first invented and published, XGBoost used to be the algorithm-of-choice for Kagglers. Lots of the winners in around the years 2015 and 2016 won their prize using XGBoost.

The support includes various objective functions, including regression, classification and ranking.

One of the most interesting things about the XGBoost is that it is also called a regularized boosting technique. This helps to reduce overfit modelling.

XGBoost is similar to gradient boosting algorithm but it has a few tricks up its sleeve which makes it stand out from the rest.

Features of XGBoost are:

- Clever Penalisation of Trees
- A Proportional shrinking of leaf nodes
- Newton Boosting
- Extra Randomisation Parameter

In XGBoost the trees can have a varying number of terminal nodes and left weights of the trees that are calculated with less evidence is shrunk more heavily. Newton Boosting uses Newton-Raphson method of approximations which provides a direct route to the minima than gradient descent. The extra randomisation parameter can be used to reduce the correlation between the trees,  the lesser the correlation among classifiers, the better our ensemble of classifiers will turn out. 

In [ ]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
clf = XGBClassifier(n_estimators = 100, max_depth = 3)
clf.fit(x_train,y_train)
clf.predict(x_test)

**Which is the best, Bagging or Boosting?**
There’s not an outright winner; it depends on the data, the simulation and the circumstances.
Bagging and Boosting decrease the variance of your single estimate as they combine several estimates from different models. So the result may be a model with higher stability.

Both are good at reducing variance and provide higher stability but only Boosting tries to reduce bias. On the other hand, Bagging may solve the over-fitting problem, while Boosting can increase it.

If the problem is that the single model gets a very low performance, Bagging will rarely get a better bias. However, Boosting could generate a combined model with lower errors as it optimises the advantages and reduces pitfalls of the single model.

By contrast, if the difficulty of the single model is over-fitting, then Bagging is the best option. Boosting for its part doesn’t help to avoid over-fitting; in fact, this technique is faced with this problem itself. For this reason, Bagging is effective more often than Boosting.

### 3 Stacking <a class="anchor" id="section_4_4_1"></a>

**Stacking:**  Trains N different weak models (usually of different types – heterogenous) with one of the two subsets of the
dataset in parallel. Once the weak learners are trained, they are used to trained a meta learner to combine their predictions and carry out final prediction using the other subset. In test phase, each model predicts its label, these set of labels are fed to the meta learner which generates the final prediction.

<img src="Images/stack.png" width="500"/>

In [ ]:
from sklearn.ensemble import StackingRegressor
models = [ ('lr', LinearRegression()),('dt', DecisionTreeRegressor()]
stacking = StackingRegressor(estimators=models, final_estimator=RandomForestRegressor(n_estimators=10,random_state=42)
stacking.fit(X_train, y_train)

<img src="Images/bbs.png" width="500"/>

## PART 2 <a class="anchor" id="chapter2"></a>

### Cross Validation <a class="anchor" id="section_4_1"></a>






**Cross-validation** is a statistical method used to estimate the performance (or accuracy) of machine learning models. It is used to protect against overfitting in a predictive model, particularly in a case where the amount of data may be limited. In cross-validation, you make a fixed number of folds (or partitions) of the data, run the analysis on each fold, and then average the overall error estimate.

<font color='Brown'>**Need of CV:**</font>

1. To Avoid Overfitting:
When we train a model on the training set, it tends to overfit most of the time, thus we utilise regularisation approaches to avoid this. Because we only have a few training instances, we must be cautious while lowering the number of training samples and conserving them for testing.

2. Support Model tuning:
Finding the best combination of model parameters is a common step to tune an algorithm toward learning the dataset’s hidden patterns. But doing this step on a simple training-testing split is typically not recommended. The model performance is usually very sensitive to such parameters and adjusting those based on a predefined dataset split should be avoided. It can cause the model to overfit and reduce its ability to generalize.

<font color='Brown'>**Types of CV:**</font>

1. Train/Test Split: Taken to one extreme, k may be set to 2 (not 1) such that a single train/test split is created to evaluate the model.
2. K-Fold Cross Validation
3. Stratified K-fold Cross-Validation
4. Leave One-out Cross Validation
5. Holdout Method

### K-Fold Cross Validation

<font color='Brown'>**How it works:**</font>


1. Pick a number of folds – K. Usually, k is 5 or 10 but you can choose any number which is less than the dataset’s length.
2. Split the dataset into k equal (if possible) parts (they are called folds)
3. Choose k – 1 folds as the training set. The remaining fold will be the test set
4. Train the model on the training set. On each iteration of cross-validation, you must train a new model independently of the model trained on the previous iteration
5. Validate on the test set and save the result
6. Repeat steps 3 – 6 *K* times. Each time use the remaining  fold as the test set. In the end, you should have validated the model on every fold that you have.


<img src="https://editor.analyticsvidhya.com/uploads/16042grid_search_cross_validation.png" width="500"/> <br>


<font color='Brown'>**Advantages of Cross-Validation :**</font>

1. Use All Your Data
2. Parameters Fine-Tuning

In [19]:
# from sklearn.model_selection import cross_val_score
# print(cross_val_score(model, X_train, y_train, cv=5))

### HyperParameter Tuning <a class="anchor" id="section_5_1"></a>

Choosing the correct set of hyperparameters to tune the models minimizes the loss function and achieves better results. 

**Model parameters:** These are the parameters that are estimated by the model from the given data. <br>
**Model hyperparameters:** These are the parameters that cannot be estimated by the model from the given data. These parameters are used to estimate the model parameters.

<font color='Brown'>**How it works:**</font>

Cross-Validation has two main steps: splitting the data into subsets (called folds) and rotating the training and validation among them. The splitting technique commonly has the following properties:

- Each fold has approximately the same size.
- Data can be randomly selected in each fold or stratified.​
- All folds are used to train the model except one, which is used for validation. That validation fold should be rotated until all folds have become a validation fold once and only once.​
- Each example is recommended to be contained in one and only one fold.​

K-fold and CV are two terms that are used interchangeably. K-fold is just describing how many folds you want to split your dataset into. Many libraries use k=10 as a default value representing 90% going to training and 10% going to the validation set. The next figure describes the process of iterating over the picked ten folds of the dataset.

<font color='Brown'>**Types of Hyperparameter tuning:**</font>  

1. **Manual:** select hyperparameters based on intuition/experience/guessing, train the model with the hyperparameters, and score on the validation data. Repeat process until you run out of patience or are satisfied with the results.
2. **Grid Search:** set up a grid of hyperparameter values and for each combination, train a model and score on the validation data. In this approach, every single combination of hyperparameters values is tried which can be very inefficient!
3. **Random search:** set up a grid of hyperparameter values and select random combinations to train the model and score. The number of search iterations is set based on time/resources.

<font color='Brown'>**Important Parameters:**</font>  

- **get_params** -->  Get parameters for this estimator.

- **cv** -->  Determines the cross-validation splitting strategy - *None*, to use the default 5-fold cross validation,

- **best_estimator_** -->  Estimator which gave highest score (or smallest loss if specified) on the left out data

- **best_score_** -->  Mean cross-validated score of the best_estimator. 

- **best_params_** -->  Parameter setting that gave the best results on the hold out data. 

#### 1. GridSearchCV <a class="anchor" id="section_5_1_1"></a>


In the grid search method, we create a grid of possible values for hyperparameters. Each iteration tries a combination of hyperparameters in a specific order. It fits the model on each combination of hyperparameters possible and records the model performance. Finally, it returns the best model with the best hyperparameters.

- **param_grid** -->  Dictionary with parameters names (str) as keys and distributions or lists of parameters to try. 

#### 2. RandomsearchCV <a class="anchor" id="section_5_1_2"></a>

In the random search method, we create a grid of possible values for hyperparameters. Each iteration tries a random combination of hyperparameters from this grid, records the performance, and lastly returns the combination of hyperparameters that provided the best performance.


- **param_distributions** -->  Dictionary with parameters names (str) as keys and distributions or lists of parameters to try.

### Model Comparison <a class="anchor" id="section_6_1"></a>

1. Time complexity

2. Space complexity

3. Sample complexity

4. Bias-variance tradeoff

5. Methodology, Assumptions and Objectives